In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy.special.orthogonal import p_roots

In [ ]:
order   = 2
L       = 2*np.pi
n_elems = 2

x_vals = np.linspace(0, L, n_elems+1)
y_vals = np.linspace(0, L/2.0, n_elems+1)

vertices = np.zeros(((n_elems+1)**2, 2), dtype=np.double)
elem_to_vertex = np.zeros((n_elems**2, 4), dtype=np.int)

for i in range(n_elems):
    for j in range(n_elems):
        elem = i*n_elems+j
        elem_to_vertex[elem,0] = i*(n_elems+1)+j
        elem_to_vertex[elem,1] = i*(n_elems+1)+j+1
        elem_to_vertex[elem,2] = (i+1)*(n_elems+1)+j+1
        elem_to_vertex[elem,3] = (i+1)*(n_elems+1)+j

for i in range(n_elems+1):
    for j in range(n_elems+1):
        v = i*(n_elems+1)+j
        vertices[v,0] = x_vals[j]
        vertices[v,1] = y_vals[i]


In [ ]:
class SQuad(object):
    
    interval = (-1.0, 1.0)
    h        = 2.0
    
    vertices = np.array([[-1,-1],
                         [1,-1],
                         [1,1],
                         [-1,1]], dtype=np.int)
    edge_to_vertex = np.array([[0,1],
                               [1,2],
                               [3,2],
                               [0,3]], dtype=np.int)
    n_vertices = 4
    n_edges    = 4
    n_vertex_per_edge = 2
    
    def calc_jacb(self, nodes):
        assert nodes.ndim==3
        
        dx = nodes[:,1,0]-nodes[:,0,0]
        dy = nodes[:,3,1]-nodes[:,0,1]
        
        jacb = np.zeros((nodes.shape[0], 2, 2))
        jacb[:,0,0] = dx/self.h
        jacb[:,1,1] = dy/self.h
        
        return jacb
        
    def calc_jacb_det(self, jacb):
        return np.linalg.det(jacb)
    
    def calc_jacb_inv(self, jacb):
        det = np.linalg.det(jacb)
        det = det.reshape((jacb.shape[0],1,1))
        inv = np.zeros_like(jacb)
        inv[:,0,0] =  jacb[:,1,1]
        inv[:,1,1] =  jacb[:,0,0]
        inv[:,0,1] = -jacb[:,0,1]
        inv[:,1,0] = -jacb[:,1,0]
        return inv/det
    
    def calc_jacb_inv_det(self, jacb_inv):
        return np.linalg.det(jacb_inv)
    
    def get_quadrature(self, n):
        xg, wg = p_roots(n)
        x = np.zeros((n**2, 2),
                     dtype=np.double)
        w = np.zeros(n**2,
                     dtype=np.double)
        
        for i in range(n):
            for j in range(n):
                p = i*n+j
                x[p, 0] = xg[j]
                x[p, 1] = xg[i]
                w[p]    = wg[i]*wg[j]
                
        return x, w
    
    def ref_to_phys(self, nodes, ref):
        pass
    
    def phys_to_ref(self, nodes, phys):
        pass
        


In [ ]:
topo  = SQuad()
nodes = vertices[elem_to_vertex]
jacb  = topo.calc_jacb(nodes)
jacb_inv = topo.calc_jacb_inv(jacb)